In [4]:
# ML modules
import pandas as pd
import numpy as np
import seaborn as sns

#GraphQl
from gql import gql, Client
from gql.dsl import DSLQuery, DSLSchema, dsl_gql
from gql.transport.requests import RequestsHTTPTransport

#API Key
from dotenv import load_dotenv
import os

# Set up logging
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [5]:
load_dotenv()
# Add your key and endpoint
key = os.getenv('API_KEY')

client_id = os.getenv('CLIENT_ID')

url_yelp = 'https://api.yelp.com/v3/graphql'

# YELP API Request

In [6]:
header = {
    'Authorization': 'bearer {}'.format(key),
    'Content-Type':"application/json"
}

transport = RequestsHTTPTransport(url = url_yelp, #Yelp.config('url'),
                                  headers = header,
                                  use_json = True)

client = Client(transport = transport, fetch_schema_from_transport = True)

cities = ['Paris', 'Lyon']

#'Paris', 'Bordeaux', 
#           'Lyon', 'Vannes',
#           'Marseille', 'Nice',
#           'Lille', 'Grenoble',
#           'Cannes', 'Limoges',
#           'Versailles', 'Nice',
#           'Nantes']

In [7]:
results = []
#query RequestForEachCity($city: String!) {

for city in cities :
    query = gql('''{
                search(term : "restaurant", location : "$city", limit : 50) {
                    business{
                        name
                        id
                        reviews {
                            id
                            text
                            rating
                        }
                    }
               }
            }''')
    
    #params = {'city': city}
    results.append(client.execute(query))#, variable_values=params))

INFO:gql.transport.requests:>>> {"query": "query IntrospectionQuery {\n  __schema {\n    queryType {\n      name\n    }\n    mutationType {\n      name\n    }\n    subscriptionType {\n      name\n    }\n    types {\n      ...FullType\n    }\n    directives {\n      name\n      description\n      locations\n      args {\n        ...InputValue\n      }\n    }\n  }\n}\n\nfragment FullType on __Type {\n  kind\n  name\n  description\n  fields(includeDeprecated: true) {\n    name\n    description\n    args {\n      ...InputValue\n    }\n    type {\n      ...TypeRef\n    }\n    isDeprecated\n    deprecationReason\n  }\n  inputFields {\n    ...InputValue\n  }\n  interfaces {\n    ...TypeRef\n  }\n  enumValues(includeDeprecated: true) {\n    name\n    description\n    isDeprecated\n    deprecationReason\n  }\n  possibleTypes {\n    ...TypeRef\n  }\n}\n\nfragment InputValue on __InputValue {\n  name\n  description\n  type {\n    ...TypeRef\n  }\n  defaultValue\n}\n\nfragment TypeRef on __Type {\

In [8]:
dict_reviews = []

for res in results:
    for val in res['search']['business']:
        for reviews in val['reviews']:
            dict_reviews.append({
                'review_id' : reviews['id'],
                'text' : reviews['text'],
                'stars' : reviews['rating']
            })

In [9]:
df_yelp = pd.DataFrame.from_dict(dict_reviews)
#df_yelp = df_yelp.dropna()

In [10]:
df_yelp

,review_id,text,stars
0,QIGpO6oIaAZui6AW9azERQ,Come here for delicious Swedish meatballs in S...,5
1,MsVIQ3WF3s0T1vjKsmuXTw,It's definitely 100x times better than ikea \...,4
2,B9OsXEcXEtsaUT10mzFvtg,Meatballs were delicious and mash was addictiv...,5
3,uj76ur3Qxqm81t-UGEHcpg,On a day trip from Norway to Sweden we had 8 h...,5
4,gLsyVW77Nyvojl-SLn2ogA,"We came in early, so there weren't too many ot...",4
...,...,...,...
282,LzJ43OmcD210rqe9CjlnEw,"Loved it. My favorite plate was the salmon, t...",5
283,u4ASeNsy5z33csctksDOTw,Imagine my determination to visit this restaur...,4
284,HoP9sTVBo8BXp2uTPj1AOw,Excellent steak. No-nonsense menu. Friendly st...,5
285,NJTJCPStM1xjGi59yl4GTw,I discovered this place through social media f...,3


In [133]:
if ~os.path.exists("data/processed/reviews_from_api.pkl.gz"):
    df_yelp.to_pickle("data/processed/reviews_from_api.pkl.gz")
    logger.info(">>> Saving reviews to pickle file that was charged from API...")

INFO:__main__:>>> Saving reviews to pickle file that was charged from API...
